In [3]:
# https://pf.mgcom.ru/task/1433882

In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from decimal import Decimal

In [6]:
#AM Orders For Matching
PATH_AM_ORDERS_FOR_MATCHING = r'C:\Rigla_BZ\2024_June\2024-06-09\Output AM\2024-06-11_170458_am_orders_for_dwh_matching_2024-06-01__2024-06-09.csv'

#DWH перед этим пересохранить файлы двх клиента, изменить источник файла на юникод utf-8 и разделитель - запятая
DWH_START_DATE = '2024-06-01'
DWH_END_DATE = '2024-06-09'
PATH_DWH_BZ = r'C:\Rigla_BZ\2024_June\2024-06-09\DWH\DWH_bz_june1-09.csv'
PATH_DWH_RIGLA = r'C:\Rigla_BZ\2024_June\2024-06-09\DWH\DWH_rigla_june1-9.csv'

#OUTPUT
PATH_OUTPUT = r'C:\Rigla_BZ\2024_June\2024-06-09\Output dwh-am'

In [7]:
def save_df_to_csv(df, dir_to_save, name):
    ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    file_name = ts + '_' + name + '.csv'
    if not os.path.exists(dir_to_save):
        os.makedirs(dir_to_save)
    csv_path = os.path.join(os.path.normpath(dir_to_save), file_name)
    df.to_csv(csv_path, index=False, sep=';', decimal=',')
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: dataFrame saved to {csv_path}")
    return

def print_ts(message):
    print("{ts}: {message}".format(ts=datetime.now().strftime('%Y-%m-%d %H:%M:%S'), message=message))
    return

# Выставляем флаги условий
def set_conditions(xdf):
    print_ts('Выставляем флаги условий')
    df = xdf.copy(deep=True)
    
    # Условия
    cond_1 = df['dwh_order_id']==df['order_id']
    # Используется диапазон дат
    cond_2 = (df['dwh_am_diff_days']>=-7) & (df['dwh_am_diff_days']<=0)
    cond_3 = df['dwh_order_sum']==df['total']
    cond_4 = df['is_reinstallation']=='true'

    # DWH совпадает с AM по Order ID
    df['dwh_am_order_id_equal'] = np.where(cond_1, 1, 0)

    # DWH совпадает с AM по Order ID и Дате
    df['dwh_am_order_id_date_equal'] = np.where(cond_1 & cond_2, 1, 0)

    # DWH совпадает с AM по Order ID и Сумме
    df['dwh_am_order_id_sum_equal'] = np.where(cond_1 & cond_3, 1, 0)

    # DWH совпадает с AM по Order ID, Дате и Сумме
    df['dwh_am_order_id_date_sum_equal'] = np.where(cond_1 & cond_2 & cond_3, 1, 0)
    
    df['am_is_reinstallation'] = np.where(cond_4, 1, 0)
    df['dwh_am_order_id_equal__am_is_reinstallation'] = np.where(cond_1 & cond_4, 1, 0)
    df['dwh_am_order_id_date_equal__am_is_reinstallation'] = np.where(cond_1 & cond_2 & cond_4, 1, 0)
    df['dwh_am_order_id_sum_equal__am_is_reinstallation'] = np.where(cond_1 & cond_3 & cond_4, 1, 0)
    df['dwh_am_order_id_date_sum_equal__am_is_reinstallation'] = np.where(cond_1 & cond_2 & cond_3 & cond_4, 1, 0)
    print_ts('OK!')
    return df

def try_decimal(x):    
    try:
        y = Decimal(x)
    except:
        y = Decimal('0')
    return y     

# Наивная валидация датф по формату и году
def validate_dates(xdate, date_format):
    try:
        date = datetime.strptime(xdate, date_format)
        valid = True if date.year == 2024 else False
    except:
        valid = False
    return valid

# Подготавливаем данные DWH
def dwh_data_prep_adhoc(xdf, date_format):
    df = xdf.copy()
    df['date_valid'] = df['create_date'].apply(lambda x: validate_dates(x, date_format))
    if(not df['date_valid'].values.all()):
        incorrect_count = df[~df['date_valid']].shape[0]
        print_ts(f'Не все даты прошли проверку. Будет удалено {incorrect_count} записей.')
        save_df_to_csv(df[~df['date_valid']], PATH_OUTPUT, f'incorrect_dates__{incorrect_count}')
        df = df[df['date_valid']]
        
    df['dwh_order_create_date'] = pd.to_datetime(df['create_date'], format=date_format)        
    df['number_brand'] = df['number'].str.split('/', expand=True)[0]
    df['dwh_order_id'] = df['number'].str.split('/', expand=True)[1]
    df['dwh_order_sum'] = df['order_sum'].fillna('0')
    df['dwh_order_sum'] = df['dwh_order_sum'].apply(lambda x: try_decimal(x))    
    df.drop_duplicates(ignore_index=True, inplace=True)
    brand_cond_v = [
        df['site_name'].str.contains('budzdorov'),
        df['site_name'].str.contains('rigla')
    ]
    brand_choises = ['budzdorov.ru', 'rigla.ru']
    df['dwh_brand'] = np.select(brand_cond_v, brand_choises, default=None)
    cols_to_use = [
        'dwh_brand',
        'dwh_order_create_date',
        'dwh_order_id',
        'dwh_order_sum'
    ]
    df = df[cols_to_use]
    return df

# Получаем данные AM (заказы для метчинга)
def get_am_data_adhoc(path):
    print_ts(f'Loading CSV - {path}')
    df = pd.read_csv(path, dtype=object)
    df.drop_duplicates(inplace=True, ignore_index=True)
    # df['event_date'] = pd.to_datetime(df['event_date'], format='%Y-%m-%d %H:%M:%S
    df['event_date'] = pd.to_datetime(df['event_date'], format='%Y-%m-%d')
    df['event_datetime'] = pd.to_datetime(df['event_date'], format='%Y-%m-%d %H:%M:%S')
    total_events = df.shape[0]
    print_ts(f'Total events: {total_events}')
    print_ts('Done!')
    return df

# Получаем данные DWH
def get_dwh_data_adhoc(path_bz, path_rigla):
    
    print_ts('Подготавливаем данные BZ')
    dwh_data_bz_raw = pd.read_csv(path_bz, sep=';', dtype=object)
    dwh_data_bz = dwh_data_prep_adhoc(dwh_data_bz_raw, "%Y-%m-%d")
    bz_events = dwh_data_bz.shape[0]
    print_ts(f'Всего событий BZ: {bz_events}')


    print_ts('Подготавливаем данные Rigla')
    dwh_data_rigla_raw = pd.read_csv(path_rigla, sep=';', dtype=object)
    dwh_data_rigla = dwh_data_prep_adhoc(dwh_data_rigla_raw, "%Y-%m-%d")
    rigla_events = dwh_data_rigla.shape[0]
    print_ts(f'Всего событий Rigla: {rigla_events}')

    # Объединяем данные
    dwh_data = pd.concat([dwh_data_bz, dwh_data_rigla])
    print_ts(f'Строк данных DWH: {dwh_data.shape[0]}')
    count_of_bz_rigla_unique = dwh_data[['dwh_brand','dwh_order_id']].drop_duplicates().shape[0]
    print_ts(f'Уникальных заказов: {count_of_bz_rigla_unique}')    

    df = dwh_data[(dwh_data['dwh_order_create_date']>=DWH_START_DATE) & (dwh_data['dwh_order_create_date']<=DWH_END_DATE)]
    df.reset_index(drop=True, inplace=True)
    print_ts('Всего записей DWH: {0}'.format(dwh_data.shape[0]))
    print_ts('Отфильтровано по дате: {0}'.format(dwh_data.shape[0] - df.shape[0]))
    print_ts('Использовано для отчётного периода записей: {0}'.format(df.shape[0]))
    print_ts('')
    print(df['dwh_order_create_date'].agg(['min','max']))    
    
    return df

# Метчим данные: к данным AM привязываем DWH
def get_am_dwh_joined_data(am_data, dwh_data):
    print_ts('Метчим данные - к данным AM привязываем DWH')
    am_dwh_orders = am_data.merge(dwh_data, how='left', left_on=['order_id','brand'], right_on=['dwh_order_id','dwh_brand'])
    am_dwh_orders['am_unique'] = 1    
    
    # Добавляем разницу в днях
    am_dwh_orders['dwh_am_diff_days'] = (am_dwh_orders['dwh_order_create_date'] - am_dwh_orders['event_date']).dt.days
    
    # Убираем лишние колонки
    useful_columns = [
        'am_unique',
        'dwh_order_create_date', 
        'dwh_order_id',
        'dwh_order_sum', 
        'dwh_brand', 
        'event_name', 
        'event_json',
        'event_datetime', 
        'city', 
        'appmetrica_device_id',
        'brand', 
        'order_id', 
        'total', 
        'publisher_name',
        'tracker_name',
        'is_reinstallation',
        'event_date',
        'event_month', 
        'dwh_am_diff_days'
    ]
    df = am_dwh_orders[useful_columns].copy()
    df['total'] = df['total'].apply(lambda x: Decimal(x))
    df['dwh_order_sum'] = df['dwh_order_sum'].apply(lambda x: Decimal(x))
    
    # Добавляем столбец с разницей по суммам
    df['dwh_am_sum_diff'] = df['dwh_order_sum'] - df['total']    
    
    print_ts('OK!')
    return df    

In [8]:
# Получаем данные AM (заказы для метчинга)
am_data_first_orders = get_am_data_adhoc(PATH_AM_ORDERS_FOR_MATCHING)

2024-06-11 17:07:55: Loading CSV - C:\Rigla_BZ\2024_June\2024-06-09\Output AM\2024-06-11_170458_am_orders_for_dwh_matching_2024-06-01__2024-06-09.csv
2024-06-11 17:07:59: Total events: 156506
2024-06-11 17:07:59: Done!


In [9]:
am_data_first_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156506 entries, 0 to 156505
Data columns (total 57 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   application_id              156506 non-null  object        
 1   app_build_number            156506 non-null  object        
 2   ios_ifa                     13936 non-null   object        
 3   ios_ifv                     52182 non-null   object        
 4   android_id                  1 non-null       object        
 5   google_aid                  96673 non-null   object        
 6   profile_id                  150001 non-null  object        
 7   os_name                     156506 non-null  object        
 8   os_version                  156506 non-null  object        
 9   device_manufacturer         156506 non-null  object        
 10  device_model                156506 non-null  object        
 11  device_type                 156506 non-

In [10]:
am_data_first_orders.iloc[0]

application_id                                                          3412045
app_build_number                                                          20275
ios_ifa                                                                     NaN
ios_ifv                                                                     NaN
android_id                                                                  NaN
google_aid                                                                  NaN
profile_id                                                              1294177
os_name                                                                 android
os_version                                                                   14
device_manufacturer                                                     Samsung
device_model                                                      Galaxy A73 5G
device_type                                                               phone
device_locale                           

In [11]:
# Получаем данные DWH
dwh_data_subset = get_dwh_data_adhoc(PATH_DWH_BZ, PATH_DWH_RIGLA)

2024-06-11 17:08:15: Подготавливаем данные BZ
2024-06-11 17:08:17: Всего событий BZ: 167438
2024-06-11 17:08:17: Подготавливаем данные Rigla
2024-06-11 17:08:18: Всего событий Rigla: 106014
2024-06-11 17:08:18: Строк данных DWH: 273452
2024-06-11 17:08:18: Уникальных заказов: 273442
2024-06-11 17:08:18: Всего записей DWH: 273452
2024-06-11 17:08:18: Отфильтровано по дате: 71
2024-06-11 17:08:18: Использовано для отчётного периода записей: 273381
2024-06-11 17:08:18: 
min   2024-06-01
max   2024-06-09
Name: dwh_order_create_date, dtype: datetime64[ns]


In [12]:
# Метчим данные: к данным AM привязываем DWH
am_dwh_orders_short = get_am_dwh_joined_data(am_data_first_orders, dwh_data_subset)

2024-06-11 17:08:32: Метчим данные - к данным AM привязываем DWH
2024-06-11 17:08:32: OK!


In [13]:
# Выставляем флаги условий
task_conds_df = set_conditions(am_dwh_orders_short)

2024-06-11 17:08:35: Выставляем флаги условий
2024-06-11 17:08:35: OK!


In [14]:
use_columns = [
    'am_unique','brand', 'publisher_name','event_month',
    'dwh_am_diff_days', 'dwh_am_order_id_equal',
    'dwh_am_order_id_date_equal', 'dwh_am_order_id_sum_equal',
    'dwh_am_order_id_date_sum_equal', 'am_is_reinstallation',
    'dwh_am_order_id_equal__am_is_reinstallation',
    'dwh_am_order_id_date_equal__am_is_reinstallation',
    'dwh_am_order_id_sum_equal__am_is_reinstallation',
    'dwh_am_order_id_date_sum_equal__am_is_reinstallation']
xdf = task_conds_df[use_columns].copy()

In [15]:
# Делаем сводную
task_conds_df_sum = xdf.groupby(['event_month', 'brand', 'publisher_name'], as_index=False, dropna=False).sum()
task_conds_df_sum.drop(['dwh_am_diff_days'], axis=1, inplace=True)

In [16]:
save_df_to_csv(task_conds_df, PATH_OUTPUT, 'task_conds')

2024-06-11 17:08:45: dataFrame saved to C:\Rigla_BZ\2024_June\2024-06-09\Output dwh-am\2024-06-11_170843_task_conds.csv


In [17]:
save_df_to_csv(task_conds_df_sum, PATH_OUTPUT, 'task_conds_agg')

2024-06-11 17:08:45: dataFrame saved to C:\Rigla_BZ\2024_June\2024-06-09\Output dwh-am\2024-06-11_170845_task_conds_agg.csv
